In [ ]:
import time
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
import re

In [ ]:
pd.options.display.width = 0
pd.set_option('display.max_columns', None)

In [ ]:
# import googlemaps

# gmaps = googlemaps.Client(key = API_KEY)

# i = 0
# for x in df2['block_street_name']:
#     while i<5:
#         i=i+1
#         #get lat/long of the place
#         print(x)
#         z = x.replace(" ", "%2C")
#         url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=" + z + "&inputtype=textquery&fields=formatted_address%2Cname%2Cgeometry&key=" + API_KEY
#         payload={}
#         headers = {}
#         response = requests.request("GET", url, headers=headers, data=payload)
#         lol = json.loads(response.text)
#         lat = str(lol['candidates'][0]['geometry']['location']['lat'])
#         lng = str(lol['candidates'][0]['geometry']['location']['lng'])

#         #get distance of amenities
#         amenities = ['hospital', 'shopping_mall']
#         print("near "+x+": ")
#         for a in amenities:
#           url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location="+lat+"%2C"+lng+"&radius=500&type="+a+"&key="+API_KEY
#           payload={}
#           headers = {}
#           response = requests.request("GET", url, headers=headers, data=payload)
#           lol2 = json.loads(response.text)
#           for y in lol2['results']:
#             print(y['name'])  
        

# Notebook Variables

In [ ]:
# OneMap Search
URL_ONEMAP = "https://developers.onemap.sg/commonapi/search"
QUERY_ONEMAP = {"returnGeom": "Y",
                "getAddrDetails": "N",
                "pageNum": 1}
PARAM_ADDRESS = "searchVal"

# Importing Data

In [ ]:
flats_raw = pd.read_csv('resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv')
flats_raw.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [ ]:
flats_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124003 entries, 0 to 124002
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                124003 non-null  object 
 1   town                 124003 non-null  object 
 2   flat_type            124003 non-null  object 
 3   block                124003 non-null  object 
 4   street_name          124003 non-null  object 
 5   storey_range         124003 non-null  object 
 6   floor_area_sqm       124003 non-null  float64
 7   flat_model           124003 non-null  object 
 8   lease_commence_date  124003 non-null  int64  
 9   remaining_lease      124003 non-null  object 
 10  resale_price         124003 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 10.4+ MB


In [ ]:
mrt_raw = pd.read_csv('mrtsg.csv')
mrt_raw.head()

,OBJECTID,STN_NAME,STN_NO,X,Y,Latitude,Longitude,COLOR
0,12,ADMIRALTY MRT STATION,NS10,24402.1063,46918.1131,1.440585,103.800998,RED
1,16,ALJUNIED MRT STATION,EW9,33518.6049,33190.0020,1.316433,103.882893,GREEN
2,33,ANG MO KIO MRT STATION,NS16,29807.2655,39105.7720,1.369933,103.849553,RED
3,81,BAKAU LRT STATION,SE3,36026.0821,41113.8766,1.388093,103.905418,OTHERS
4,80,BANGKIT LRT STATION,BP9,21248.2460,40220.9693,1.380018,103.772667,OTHERS


In [ ]:
mrt_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   OBJECTID   189 non-null    int64  
 1   STN_NAME   189 non-null    object 
 2   STN_NO     189 non-null    object 
 3   X          189 non-null    float64
 4   Y          189 non-null    float64
 5   Latitude   189 non-null    float64
 6   Longitude  189 non-null    float64
 7   COLOR      189 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 11.9+ KB


In [ ]:
hospitals_raw = pd.read_csv('hospitals.csv', names=["Name", "Address"])
hospitals_raw.head(5)

,Name,Address
0,ALEXANDRA HOSPITAL,378 ALEXANDRA ROAD Singapore 159964
1,ANG MO KIO - THYE HUA KWAN HOSPITAL,17 ANG MO KIO AVENUE 9 Singapore 569766
2,BRIGHT VISION HOSPITAL,5 LORONG NAPIRI Singapore 547530
3,CHANGI GENERAL HOSPITAL,2 SIMEI STREET 3 Singapore 529889
4,COMPLEX MEDICAL CENTRE,982 UPPER CHANGI ROAD NORTH Singapore 507709


In [ ]:
hospitals_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     30 non-null     object
 1   Address  30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


In [ ]:
schools_raw = pd.read_csv('general-information-of-schools.csv', dtype={'postal_code': object})
schools_raw.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,bus_desc,principal_name,first_vp_name,second_vp_name,third_vp_name,fourth_vp_name,fifth_vp_name,sixth_vp_name,dgp_code,zone_code,type_code,nature_code,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code
0,ADMIRALTY PRIMARY SCHOOL,https://admiraltypri.moe.edu.sg/,11 WOODLANDS CIRCLE,738907,63620598,na,63627512,na,ADMIRALTY_PS@MOE.EDU.SG,Admiralty Station,"TIBS 965, 964, 913",MR PEK WEE HAUR,MDM CHUA MUI LING,MDM NUR SABARIAH BTE MOHD IBRAHIM,NaN,NaN,NaN,NaN,WOODLANDS,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,FULL DAY,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
1,ADMIRALTY SECONDARY SCHOOL,http://www.admiraltysec.moe.edu.sg,31 WOODLANDS CRESCENT,737916,63651733,63654596,63652774,na,Admiralty_SS@moe.edu.sg,ADMIRALTY MRT,904,MR LAM YUI- P'NG,MR NG SONG LIM STEVEN,MR SHEIK ALAUDIN B MOHD ISMAIL,NaN,NaN,NaN,NaN,WOODLANDS,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
2,AHMAD IBRAHIM PRIMARY SCHOOL,http://www.ahmadibrahimpri.moe.edu.sg,10 YISHUN STREET 11,768643,67592906,na,67592927,na,aips@moe.edu.sg,Yishun,Yishun Ring Road - 812 (white plate); Yishun A...,MISS ONG LEE CHOO,MADAM SULASNI BINTE ABDUL RAHMAN,NaN,NaN,NaN,NaN,NaN,YISHUN,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil
3,AHMAD IBRAHIM SECONDARY SCHOOL,http://www.ahmadibrahimsec.moe.edu.sg,751 YISHUN AVENUE 7,768928,67585384,na,67557778,na,aiss@moe.edu.sg,"CANBERRA MRT, YISHUN MRT","117, 167, 169, 800, 811, 812, 883, 856, 858, 8...",MISS TAN KE XIN,MR YAO ZHIXUAN TERENCE,MISS LEA SHU HUI,NaN,NaN,NaN,NaN,YISHUN,NORTH,GOVERNMENT SCHOOL,CO-ED SCHOOL,SINGLE SESSION,SECONDARY,No,No,No,No,Chinese,Malay,Tamil
4,AI TONG SCHOOL,http://www.aitong.moe.edu.sg,100 Bright Hill Drive,579646,64547672,na,64532726,na,AITONG_SCH@MOE.EDU.SG,Bishan MRT,"410, 162, 52",MDM GOH LING LING DIANE,"MDM NG BEE ENG, CINDY",MR YAK BENG WEI,NaN,NaN,NaN,NaN,BISHAN,SOUTH,GOVERNMENT-AIDED SCH,CO-ED SCHOOL,SINGLE SESSION,PRIMARY,Yes,No,No,No,Chinese,na,na


In [ ]:
schools_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   school_name         346 non-null    object
 1   url_address         346 non-null    object
 2   address             346 non-null    object
 3   postal_code         346 non-null    object
 4   telephone_no        346 non-null    object
 5   telephone_no_2      346 non-null    object
 6   fax_no              346 non-null    object
 7   fax_no_2            346 non-null    object
 8   email_address       346 non-null    object
 9   mrt_desc            346 non-null    object
 10  bus_desc            346 non-null    object
 11  principal_name      346 non-null    object
 12  first_vp_name       346 non-null    object
 13  second_vp_name      237 non-null    object
 14  third_vp_name       13 non-null     object
 15  fourth_vp_name      5 non-null      object
 16  fifth_vp_name       2 non-

# Clean Data

In [ ]:
flats = flats_raw.copy()
mrt = mrt_raw.copy()
hospitals = hospitals_raw.copy()

In [ ]:
# Aggregating flats by address
flats = flats.groupby(["street_name", "block"]).count().reset_index()
flats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9216 entries, 0 to 9215
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   street_name          9216 non-null   object
 1   block                9216 non-null   object
 2   month                9216 non-null   int64 
 3   town                 9216 non-null   int64 
 4   flat_type            9216 non-null   int64 
 5   storey_range         9216 non-null   int64 
 6   floor_area_sqm       9216 non-null   int64 
 7   flat_model           9216 non-null   int64 
 8   lease_commence_date  9216 non-null   int64 
 9   remaining_lease      9216 non-null   int64 
 10  resale_price         9216 non-null   int64 
dtypes: int64(9), object(2)
memory usage: 792.1+ KB


In [ ]:
flats["address"] = flats["street_name"] + " " + flats["block"]
flats.head()

,street_name,block,month,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,address
0,ADMIRALTY DR,353A,15,15,15,15,15,15,15,15,15,ADMIRALTY DR 353A
1,ADMIRALTY DR,353B,14,14,14,14,14,14,14,14,14,ADMIRALTY DR 353B
2,ADMIRALTY DR,353C,17,17,17,17,17,17,17,17,17,ADMIRALTY DR 353C
3,ADMIRALTY DR,354A,12,12,12,12,12,12,12,12,12,ADMIRALTY DR 354A
4,ADMIRALTY DR,354B,11,11,11,11,11,11,11,11,11,ADMIRALTY DR 354B


In [ ]:
# Querying OneMap API for flats (commented out because this is a very expensive and time-consuming operation)

# flats["lat"] = 0.0
# flats["long"] = 0.0
# for index,row in flats.iterrows():
#   if index % 10 == 0:
#     print(index)
#   QUERY_ONEMAP[PARAM_ADDRESS] = row["address"]
#   response = requests.get(URL_ONEMAP, params=QUERY_ONEMAP)
#   if len(response.json()["results"]):
#     flats.loc[index, "lat"] = response.json()["results"][0]["LATITUDE"]
#     flats.loc[index, "long"] = response.json()["results"][0]["LONGITUDE"]
# flats.head()

In [ ]:
mrt.groupby("STN_NAME").count().sort_values("OBJECTID", ascending=False)
# MRT rows are not unique

,OBJECTID,STN_NO,X,Y,Latitude,Longitude,COLOR
STN_NAME,,,,,,,
DHOBY GHAUT MRT STATION,3,3,3,3,3,3,3
BISHAN MRT STATION,2,2,2,2,2,2,2
TAMPINES MRT STATION,2,2,2,2,2,2,2
PROMENADE MRT STATION,2,2,2,2,2,2,2
JURONG EAST MRT STATION,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
HILLVIEW MRT STATION,1,1,1,1,1,1,1
HOLLAND VILLAGE MRT STATION,1,1,1,1,1,1,1
HOUGANG MRT STATION,1,1,1,1,1,1,1


In [ ]:
mrt = mrt.groupby("STN_NAME").agg({"Latitude":'mean', "Longitude":'mean', "STN_NO": "count"}).reset_index()
mrt = mrt.rename(columns={"STN_NAME": "station_name", "Latitude": "lat", "Longitude": "long", "STN_NO": "number_lines"})
mrt.sort_values("number_lines", ascending=False).head(10)

,station_name,lat,long,number_lines
45,DHOBY GHAUT MRT STATION,1.299083,103.845944,3
13,BISHAN MRT STATION,1.351074,103.848645,2
140,TAMPINES MRT STATION,1.354189,103.944074,2
110,PROMENADE MRT STATION,1.293055,103.860919,2
67,JURONG EAST MRT STATION,1.333153,103.742311,2
36,CITY HALL MRT STATION,1.292936,103.852581,2
26,BUONA VISTA MRT STATION,1.306838,103.790434,2
50,EXPO MRT STATION,1.334966,103.961927,2
115,RAFFLES PLACE MRT STATION,1.284126,103.851457,2
104,PAYA LEBAR MRT STATION,1.317771,103.892604,2


In [ ]:
hospitals["postal_code"] = hospitals["Address"].apply(lambda x: re.search(r"Singapore [0-9]{6}", x, flags=re.I).group())
hospitals.head()

,Name,Address,lat,long,postal_code
0,ALEXANDRA HOSPITAL,378 ALEXANDRA ROAD Singapore 159964,1.28859804100324,103.799973882445,Singapore 159964
1,ANG MO KIO - THYE HUA KWAN HOSPITAL,17 ANG MO KIO AVENUE 9 Singapore 569766,1.38401946148579,103.840355932686,Singapore 569766
2,BRIGHT VISION HOSPITAL,5 LORONG NAPIRI Singapore 547530,0.0,0.0,Singapore 547530
3,CHANGI GENERAL HOSPITAL,2 SIMEI STREET 3 Singapore 529889,1.34053138453855,103.949633084403,Singapore 529889
4,COMPLEX MEDICAL CENTRE,982 UPPER CHANGI ROAD NORTH Singapore 507709,1.35790230865333,103.971308670089,Singapore 507709


In [ ]:
# Querying OneMap API for hospitals (commented out because this is a very expensive and time-consuming operation)

# hospitals["lat"] = 0.0
# hospitals["long"] = 0.0
# for index,row in hospitals.iterrows():
#   print(index, row["Address"])
#   QUERY_ONEMAP[PARAM_ADDRESS] = row["postal_code"]
#   response = requests.get(URL_ONEMAP, params=QUERY_ONEMAP)
#   if len(response.json()["results"]):
#     hospitals.loc[index, "lat"] = response.json()["results"][0]["LATITUDE"]
#     hospitals.loc[index, "long"] = response.json()["results"][0]["LONGITUDE"]
# hospitals.head(30)

0  378 ALEXANDRA ROAD Singapore 159964
1  17 ANG MO KIO AVENUE 9 Singapore 569766
2  5 LORONG NAPIRI Singapore 547530
3  2 SIMEI STREET 3 Singapore 529889
4  982 UPPER CHANGI ROAD NORTH Singapore 507709
5  19 ADAM ROAD Singapore 289891
6  1 FARRER PARK STATION ROAD Singapore 217562
7  6A Napier Road Singapore 258500
8  10 BUANGKOK VIEW Singapore 539747
9  1 JURONG EAST STREET 21 Singapore 609606
10  90 YISHUN CENTRAL Singapore 768828
11  100 BUKIT TIMAH ROAD Singapore 229899
12  820 THOMSON ROAD Singapore 574623
13  3 Mount Elizabeth Singapore 228510
14  38 Irrawaddy Road Singapore 329563
15  5 HOSPITAL DRIVE Singapore 169609
16  5 LOWER KENT RIDGE ROAD Singapore 119074
17  1 JURONG EAST STREET 21 Singapore 609606
18  10 HOSPITAL BOULEVARD Singapore 168582
19  321 JOO CHIAT PLACE Singapore 427990
20  585 NORTH BRIDGE ROAD Singapore 188770
21  71 IRRAWADDY ROAD Singapore 329562
22  1 ANCHORVALE STREET Singapore 544835
23  110 Sengkang East Way Singapore 544886
24  Outram Road Singapore 

,Name,Address,lat,long,postal_code
0,ALEXANDRA HOSPITAL,378 ALEXANDRA ROAD Singapore 159964,1.28859804100324,103.799973882445,Singapore 159964
1,ANG MO KIO - THYE HUA KWAN HOSPITAL,17 ANG MO KIO AVENUE 9 Singapore 569766,1.38401946148579,103.840355932686,Singapore 569766
2,BRIGHT VISION HOSPITAL,5 LORONG NAPIRI Singapore 547530,1.37194604127942,103.878032898695,Singapore 547530
3,CHANGI GENERAL HOSPITAL,2 SIMEI STREET 3 Singapore 529889,1.34053138453855,103.949633084403,Singapore 529889
4,COMPLEX MEDICAL CENTRE,982 UPPER CHANGI ROAD NORTH Singapore 507709,1.35790230865333,103.971308670089,Singapore 507709
5,CRAWFURD HOSPITAL,19 ADAM ROAD Singapore 289891,1.32680172823489,103.813322242685,Singapore 289891
6,FARRER PARK HOSPITAL,1 FARRER PARK STATION ROAD Singapore 217562,1.31264476602917,103.854018095117,Singapore 217562
7,GLENEAGLES HOSPITAL,6A Napier Road Singapore 258500,1.30736473510568,103.819420889381,Singapore 258500
8,INSTITUTE OF MENTAL HEALTH / WOODBRIDGE HOSPITAL,10 BUANGKOK VIEW Singapore 539747,1.38168746675049,103.884323802695,Singapore 539747
9,JURONG COMMUNITY HOSPITAL,1 JURONG EAST STREET 21 Singapore 609606,1.33553382554428,103.743865452312,Singapore 609606


In [ ]:
schools = schools_raw[["school_name", "address", "postal_code", "mainlevel_code"]].copy().rename(columns={"mainlevel_code": "school_type"})
schools.head()

,school_name,address,postal_code,school_type
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,PRIMARY
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,SECONDARY
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,PRIMARY
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,SECONDARY
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,PRIMARY


In [ ]:
# Querying OneMap API for schools

schools["lat"] = 0.0
schools["long"] = 0.0
for index,row in schools.iterrows():
  if index % 25 == 0:
    print(index)
  QUERY_ONEMAP[PARAM_ADDRESS] = row["postal_code"]
  response = requests.get(URL_ONEMAP, params=QUERY_ONEMAP)
  if len(response.json()["results"]):
    schools.loc[index, "lat"] = response.json()["results"][0]["LATITUDE"]
    schools.loc[index, "long"] = response.json()["results"][0]["LONGITUDE"]
  else:
    print(f'latlong for {row["postal_code"]} at index: {index} cannot be found')
schools.head()

0
25
50
75
100
125
150
175
200
225
250
275
300
325


,school_name,address,postal_code,mainlevel_code,lat,long
0,ADMIRALTY PRIMARY SCHOOL,11 WOODLANDS CIRCLE,738907,PRIMARY,1.44298589789841,103.800304550731
1,ADMIRALTY SECONDARY SCHOOL,31 WOODLANDS CRESCENT,737916,SECONDARY,1.44589068910993,103.802398196596
2,AHMAD IBRAHIM PRIMARY SCHOOL,10 YISHUN STREET 11,768643,PRIMARY,1.43368120674913,103.832924289687
3,AHMAD IBRAHIM SECONDARY SCHOOL,751 YISHUN AVENUE 7,768928,SECONDARY,1.43623514612253,103.82998645047
4,AI TONG SCHOOL,100 Bright Hill Drive,579646,PRIMARY,1.36073636488519,103.833075654586


# Exporting Data

In [ ]:
flats.to_csv("flats_latlong.csv")

In [ ]:
mrt.to_csv("mrt_latlong.csv")

In [ ]:
hospitals.to_csv("hospitals_latlong.csv")

In [ ]:
schools.to_csv("schools_latlong.csv")

# Appendix

In [ ]:
# Converting month to date
flats = flats_raw.copy()
flats["month"] = pd.to_datetime(flats["month"], format='%Y-%M')
flats.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01-01 00:01:00,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01-01 00:01:00,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01-01 00:01:00,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01-01 00:01:00,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01-01 00:01:00,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0
